In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup
import collection.connect_mongodb as connect_mongodb
import time
from pymongo import errors
from IPython.display import clear_output
import concurrent.futures

In [2]:
def update_json(file_path, new_data):
    
    with open(file_path,'r',encoding = 'utf-8') as file:
        if os.path.exists(file_path):
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = {}
        else:
            data = {}
    if not isinstance(data,dict):
        data ={}
    data.update(new_data)
    
    with open(file_path,'w',encoding='utf_8') as file:
        json.dump(data,file,ensure_ascii=False,indent=4)

In [3]:
product_photo = {}
file_path = "json_photo.json"
with open(file_path,'r',encoding='utf-8') as file:
    try:
        product_photo = json.load(file)
    except:
        product_photo = {}
        
print(len(product_photo))
product_error = {}
file_path = "json_error.json"
with open(file_path,'r',encoding='utf-8') as file:
    try:
        product_error = json.load(file)
    except:
        product_error = {}
        
print(len(product_error))

30
3


In [4]:
def download_image(link_image,link,product_id,url_link):
    global product_error
    global product_photo
    try:
        api_response = requests.get(link)
        api_response.raise_for_status()
        if api_response.status_code ==200:
            folder = r"D:\project_glamira\crawl_data\crawl_photo\photo"
            dest_file = os.path.join(folder,link_image)
            with open(dest_file,'wb') as file:
                file.write(api_response.content)
        else:
            print("ERROR")
            product_error[product_id] = url_link
            update_json('json_error.json',product_error)
    except Exception as e:
        print("Fail to fetch data:",e)
        product_error[product_id] = url_link
        update_json('json_error.json',product_error)

In [5]:
import requests.adapters


def get_product_photo(url_link,product_id):
    global product_photo
    global product_error
    try:
        session = requests.Session()
        retry_strategy = requests.packages.urllib3.util.retry.Retry(
            total = 3,
            status_forcelist = [429,500,502,503,504],
            allowed_methods = ["HEAD","GET","OPTION"]
        )
        adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://",adapter)
        session.mount("http://",adapter)
        
        api_response = session.get(url_link,timeout=10)
        api_response.raise_for_status()
        if api_response.status_code ==200:
            content = api_response.content
            soup = BeautifulSoup(content,'html.parser')
            try:
                gallery = soup.find_all('div', class_ =lambda x: x and'gallery-item' in x)
                i=1
                gallery_photo = []
                for gallery_item in gallery:
                    image = gallery_item.find('img')
                    srcset = image['srcset']
                    link = srcset.split()[2]
                    link_image =str(product_id)+"_image_"+str(i)+".jpg"
                    i+=1
                    download_image(link_image,link,product_id,url_link)
                    gallery_photo.append(link_image)
                    # print(link)
                product_photo[product_id] =gallery_photo
                file_path = 'json_photo.json'
                update_json(file_path,product_photo)
                # print("update success")
                return product_photo
            except AttributeError:
                print("Don have photo")
                product_error[product_id] = url_link
                update_json('json_error.json',product_error)
                return {}
        else:
            print("Fail to fetch data")
            product_error[product_id] = url_link
            update_json('json_error.json',product_error)
            return {}
    except Exception as e:
        print("Fail to fetch data:",e)
        product_error[product_id] = url_link
        update_json('json_error.json',product_error)
        return {}

In [6]:
def main_code():
    collection =connect_mongodb.connect_db()
    documents = collection.find({}, {"product_id":1,'current_url': 1, '_id': 0})
    doc = 0
    for document in documents:
        doc+=1
        print(f"document:{doc}")
        if(doc%50==0):
            file_path = 'json_photo.json'
            update_json(file_path,product_photo)
            print("Sleep 5s")
            time.sleep(5)
            clear_output(wait=True)
        product_id = document.get('product_id')
        if product_id not in product_photo and product_id not in product_error  :
            current_url = document.get('current_url')
            if 'product_id' in document  and 'www.glamira' in current_url:
                print(current_url)
                print(product_id)
                get_product_photo(current_url,product_id)   
            print(f"product_dict_len: {len(product_photo)}")
        print("--------------------------------------------------------")
    # with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    #     futures = [executor.submit(get_product_name,document.get('current_url'),document.get('product_id')) for document in documents if  'product_id' in document]
        
    #     for future in concurrent.futures.as_completed(futures):
    #         result = future.result()
    #         print(f"len of product dictionary:{len(product_name_dicts)}")

In [7]:
def run_code():
    collection =connect_mongodb.connect_db()
    documents = collection.find({}, {"product_id":1,'current_url': 1, '_id': 0})
    doc = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_product_photo,document.get('current_url'),document.get('product_id')) for document in documents if  document.get('product_id') not in product_photo and document.get('product_id') not in product_error and 'product_id' in document  and 'www.glamira' in document.get('current_url')]
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                print(f"len of product dictionary:{len(product_photo)}")
    

In [8]:
run_code()

Fail to fetch data: 404 Client Error: Not Found for url: https://www.glamira.com.tr/glamira-ring-jenita.html?alloy=white-585&diamond=blackdiamond&stone2=diamond-Brillant
https://www.glamira.com.au/media/product/newgeneration/view/1/sku/WSR-28/alloycolour/red_white.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/1/sku/anni-n-036/diamond/diamond-Brillant_AAA/stone2/diamond-Brillant_AAA/stone3/diamond-Swarovsky_AAAAA/alloycolour/white.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/1/sku/sunita-n/diamond/diamond-Brillant_AAA/alloycolour/white_red.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/1/sku/KIDB-U022/diamond/ruby_AAA/alloycolour/white.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/1/sku/amanda2-n/diamond/emerald_AA/alloycolour/white_red.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/2/sku/WSR-28/alloycolour/red_white.jpg
https://cdn-media.glamira.com/media/product/newgeneration/view/1